In [1]:
import pandas as pd
import sqlite3
import sqlalchemy

In [75]:
#source: https://www.nature.com/articles/s41562-020-0909-7

#filter low level changes and duplicate descriptions
context= pd.read_csv("./coronanet_release_allvars.csv", low_memory=False)
context = context[context['entry_type']!="update"]
context = context[context['correct_type']!="correction"]
context = context[context['init_country_level']=="National"] #ignore regional
context = context[~context['country'].str.contains(",")] #ignore measures implemented by multiple countries at once


context.drop_duplicates(subset ="description", keep = False, inplace = True)

#drop not needed cols
context = context.drop(['record_id', 'policy_id', 'date_end', 'date_announced', 'enforcer', 'dist_index_high_est', 'dist_index_med_est', 'dist_index_low_est', 'dist_index_country_rank', 'institution_status', 'target_country', 'target_geog_level', 'target_region', 'target_province', 'target_city', 'target_other', 'target_who_what', 'target_direction', 'link', 'date_updated', 'recorded_date', 'Rank_FP', 'Score_FP', 'muni_IDC', 'dispersive_IDC', 'constraining_IDC', 'sfi_SFI', 'ti_cpi_TI', 'pop_WDI_PW', 'gdp_WDI_PW', 'growth_WDI_PW', 'lnpop_WDI_PW', 'lngdp_WDI_PW', 'disap_FA', 'polpris_FA', 'latentmean_FA', 'transparencyindex_HR', 'EmigrantStock_EMS', 'type_sub_cat'], axis = 1)
context = context.drop(['tests_units', 'total_vaccinations', 'daily_vaccinations', 'total_vaccinations_per_hundred', 'daily_vaccinations_per_million','confirmed_cases', 'deaths', 'recovered', 'new_tests', 'total_tests', 'total_tests_per_thousand'], axis = 1)
context = context.drop(['province', 'ISO_L2','ISO_A3','ISO_A2', 'city'], axis = 1)
context = context.drop(['update_level', 'correct_type', 'travel_mechanism', 'compliance', 'entry_type', 'update_type', 'type_text', 'init_country_level', 'domestic_policy'], axis = 1)

#bring to shape (similar to memories)
context = context.rename(columns={"date_start": "date", "type": "category", "description": "comment"})
context['date'] = pd.to_datetime(context['date'], format='%Y/%m/%d')
context['date'] = context['date'].dt.date
context["weight"] = 8

In [76]:
context

,comment,date,country,category,weight
0,"March 6, Afghanistan ""Measures have been taken into account to dispel disinformation and superstitions around COVID-19, Mr Mayar spokesperson for the Ministry of Public Health said that a cross-sectoral coordination committee has been formed under the chairmanship of the Minister of Public Health and with the participation of the Ministries of Hajj and Endowments and Ministries of Foreign Affairs and Interior. The Ministry of Hajj and Endowments will inform the public about coronavirus at friday prayers to dispel religious superstitions about coronavirus""",2020-03-06,Afghanistan,Anti-Disinformation Measures,8
1,"March 16, In Afghanistan the Government Media Information Center has called on telecommunication companies to put warning signs on false messages to prevent misinformation",2020-03-16,Afghanistan,Anti-Disinformation Measures,8
2,"Afghanistan, May 28 ""a government livelihood support program, Targeting the Ultra Poor (TUP) for impoverished communities has been scaled up to fight misinformation and help curb the spread of COVID-19.""",2020-05-28,Afghanistan,Anti-Disinformation Measures,8
3,"Afghanistan, June 28 ""Public awareness campaigns launched by Afghan government's Citizen’s Charter program. The campaigns have helped dispel misinformation while promoting health recommendations and medical guidelines set by the Ministry of Public Health.""",2020-06-28,Afghanistan,Anti-Disinformation Measures,8
36,"Aug 4, Afghanistan ""has announced that all private and public universities will reopen tomorrow (Aug 5). Seqiq Sediqi, spokesman for the Presidential Palace, told a press conference in Kabul that the plan of restarting universities by the Ministry of Higher Education was accepted by the Cabinet at today meeting. He said all universities were urged to seriously consider and follow the guidelines by the Ministry of Public Health in order to prevent the spread of the virus.""",2020-08-05,Afghanistan,Closure and Regulation of Schools,8
...,...,...,...,...,...
74107,"The Zimbabwean minister for health, who is also the vice president for the country, Dr Chiwenga has announced the cancellation of the scheduled By-elections. These elections have been cancelled as a public health precaution to contain the risks of the spread of covid-19.",2020-10-02,Zimbabwe,Restriction and Regulation of Government Services,8
74108,"A presidential amnesty was announced by the Zimbabwe government on March 2, allowing for the release of up to 6,000 prisoners to ease pressure on overcrowded jails. Amnesty will be given to prisoners convicted of non-violent crimes.",2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,8
74109,"In Zimbabwe, the Zimbabwe International Trade Fair (ZITF) that was to take place in the south-western city of Bulawayo from 21-25 April has been postponed, as announced on the 17th of March.",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,8
74110,"In Zimbabwe, the 40th independence anniversary celebrations that were to take place in the south-western city of Bulawayo on the 18th of April have been postponed, as announced on the 17th of March.",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,8


In [77]:
#SQLTE
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
context.to_sql(con=sqlite, name='contexts', index=True, if_exists='replace',dtype={"date": str(sqlalchemy.types.DateTime())})